# New tasks:
- make a function that read a fasta file from disk and yields (header, seq) pairs +
- ex from:
>AB003409.1/96-167
GGGCCCAUAGCUCAGUGGUAGAGUGCCUCCUUUGCAAGGAGGAUGCCCUGGGUUCGAAUC comment
CCAGUGGGUCCA
>AB009835.1/1-71
CAUUAGAUGACUGAAAGCAAGUACUGGUCUCUUAAACCAUUUAAUAGUAAAUacagugcCUU
CAUUAGAUGACUGAAAGCAAGUACUGGUCUCUUAAACCAUUUAAUAGUAAAUacagugcCUU
CAUUAGAUGACUGAAAGCAAGUACUGGUCUCUUAAACCAUUUAAUAGUAAAUacagugcCUU
CAUUAGAUGACUGAAAGCAAGUACUGGUCUCUUAAACCAUUUAAUAGUAAAUacagugcCUU
- yield:
(AB003409.1/96-167, GGGCCCAUAGCUCAGUGGUAGAGUGCCUCCUUUGCAAGGAGGAUGCCCUGGGUUCGAAUCCCAGUGGGUCCA)
(AB009835.1/1-71,CAUUAGAUGACUGAAAGCAAGUACUGGUCUCUUAAACCAUUUAAUAGUAAAUacagugcCUUCAUUAGAUGACUGAAAGCAAGUACUGGUCUCUUAAACCAUUUAAUAGUAAAUacagugcCUUCAUUAGAUGACUGAAAGCAAGUACUGGUCUCUUAAACCAUUUAAUAGUAAAUacagugcCUUCAUUAGAUGACUGAAAGCAAGUACUGGUCUCUUAAACCAUUUAAUAGUAAAUacagugcCUU)

- make a function that receives in input the list of sequences, and yields structure graphs +
- make a function that receives a iterator over graphs and i,j and plots only graphs from num i to num j


1. test if space or tab
2. test if command (after a space or any latter exept 'aucg' or 'rnytkmswbdhvgu')
3. test if new line without header then concatenate
4. test if new line with header then new graph

In [84]:
%matplotlib inline
import os, sys
import subprocess as sp
from itertools import cycle
import networkx as nx
import re
from eden.util import display

class FastaToGraph(object):
    def __init__(self, mode='RNAfold'):
        self.mode = mode
    
    def SequeceWrapper(self, filePath):
        #path = "/home/alsheikm/Work/EDeN_examples/fastaFiles/"
        seq = self._readFastaFile(filePath)
        seqs = self._removeCommends(seq)
        seqFold, strucFold = self._get_sequence_structure(seqs)
        #print ("path", path)
        return seq
    
# read a fasta file separate the head and the sequence
    def _readFastaFile(self, filePath):
        head_start = '>'
        heads = []
        seqs = []
        seq_temps = []
        string_seq = ''        

        #for file in os.listdir(path): #open file
        read_file = open(filePath,'r')            
            # seperate the head and the sequence
        for line in read_file:
            lines = list(line)
                # the read line is the head of the sequence write it in head list
            if lines[0] == head_start:
                line = line.split(head_start)
                heads.append(line)
                seqs.append(string_seq)
                seq_temps = []

                # the read line is a sequence writ it in a sequence list
            if lines[0] != head_start:
                seq_temps.append(line)
                string_seq= ''.join(seq_temps)

        seqs.append(string_seq)      
        print ('heads', heads)
        print ("seq", seqs)            
        return seqs
    
    #read a line ignore string after the whitspace, tab and not (a,c,g,u)
    def _removeCommends(self, seq):
        not_added_string = []
        input_seqs = seq
        sequences = []
        seqs = []
        
        for seq in input_seqs:
            seqs1 = []
            for i, letter in enumerate(seq):                 
                if letter == "A" or letter == "C" or letter == "G" or letter == "U":
                    seq_list = []
                    seq_list = ''.join(letter)
                    seqs1.append(seq_list)
                    
                else:
                    if letter == " " or letter != "A" or letter != "C" or letter != "G" or letter != "U":
                        not_added_string.append(letter)
            seqs.append(seqs1)
        
        for seq in seqs:
            string_seqs = ''
            string_seqs = ''.join(seq)
            sequences.append(string_seqs)
            
        print ("sequences", sequences)     
        print ('final not_added:', not_added_string)
        return sequences
    
    #call RNAfold
    def _get_sequence_structure(self, seqs):
        print "im here"
        if self.mode == 'RNAfold':
            return self._rnafold_wrapper(seqs)
        else:
            raise Exception('Not known: %s'% self.mode)
    
    def _rnafold_wrapper(self, sequence):
        print ("sequence", sequence)
        if sequence != '':
            flags='--noPS'
            cmd = 'echo "%s" | RNAfold %s' % (sequence, flags)
            out = sp.check_output(cmd, shell=True)
            text = out.strip().split('\n')
            print ("text",text)
            seq= text[0]
            print("seq",seq)
            struc= text[1].split()[0]
            print ("strc", struc)
            return seq, struc

    #Recognize basepairs and add them to the generated graph
    def _make_graph(self, seqFold, strucFold):
            print ("seq", seq)
            print ("struc", struc)
            open_pran = "("
            close_pran = ")"
            stack_o = []
            stack_c = []
            G = nx.Graph()
            seq_struc_zip = zip(seq, struc) 
            #print seq_struc_zip
            for i, k in enumerate(struc):
                G.add_node(i, label = seq[i])
                if i > 0:
                    G.add_edge(i-1, i, label= 'x')
                    if struc[i] == open_pran:
                        j = i
                        stack_o.append(struc[j])
                    if struc[i] == close_pran:
                        stack_c.append(struc[i])
                        stack_o.pop()
                        j= j-1
                        stack_c.pop()
                        G.add_edge(i, j, label = 'b')
            return G

##################################################

#generate the graph for each read sequence
    #def _makeFastaGraph(self, seq):
     #   G = nx.Graph()
        #for j,sequence in enumerate(seq):
    #        print ('j', j)
          #  print ("sequence", sequence)
           # sequences = list(sequence)
            #for i,letter in enumerate(sequences):
             #   G.add_node(i, label = sequences[i])
                #if i > 0:
                 #   G.add_edge(i-1, i, label = 'x')
            #return G
#########################################################

     

# Experiment

In [83]:
#Read fasta files and separate sequences from the head of the sequence
fgraph = FastaToGraph(mode= 'RNAfold')
path = "/home/alshaikm/EDeN_examples/fasta/testfasta"
seqs = fgraph.SequeceWrapper(path)

('heads', [['', 'AB003409.1/96-167\n'], ['', 'AB009835.1/1-71\n']])
('seq', ['', 'GGGCCCAUAGCUCAGUGGUAGAGUGCCUCCUUUGCAAGGAGGAUGCCCUGGGUUCGAAUC comments\nCCAGUGGGUCCA\n', 'CAUUAGAUGACUGAAAGCAAGUACUGGUCUCUUAAACCAUUUAAUAGUAAAUUAGCACUU\nACUUCUAAUGA\n'])
('sequences', ['', 'GGGCCCAUAGCUCAGUGGUAGAGUGCCUCCUUUGCAAGGAGGAUGCCCUGGGUUCGAAUCCCAGUGGGUCCA', 'CAUUAGAUGACUGAAAGCAAGUACUGGUCUCUUAAACCAUUUAAUAGUAAAUUAGCACUUACUUCUAAUGA'])
('final not_added:', [' ', 'c', 'o', 'm', 'm', 'e', 'n', 't', 's', '\n', '\n', '\n', '\n'])
im here
('text', [''])
('seq', '')


IndexError: list index out of range

In [67]:
#generating the graph
for seq in seqs:
    if seq != '':
        G = fgraph._make_Graph(seq)
        display.draw_graph(G, node_size=180, font_size=9, node_border=True, prog='neato')

AttributeError: 'FastaToGraph' object has no attribute '_make_Graph'